In [1]:
from bs4 import BeautifulSoup as bs
from seleniumrequests import Chrome
import re
from unidecode import unidecode
from core.models import *
from django import db
import datetime



def scrapper_2_instancia(progressivo=True):

    def calculo_cnj(seq, ano, j, tr, vara):
        vara = str(vara)
        while len(vara) < 4:
            vara = vara + '0'
        n = int(f'{seq}{ano}{j}{tr}{vara}00')
        d = str(98 - n%97)
        if len(d) == 1:
            d = '0' + d
        seq = str(seq)
        while len(seq) < 7:
            seq = '0' + seq
        return f'{seq}-{d}.{ano}.{j}.{tr}.{vara}'


    driver = Chrome()

    driver.get('https://esaj.tjsp.jus.br/sajcas/login?service=https%3A%2F%2Fesaj.tjsp.jus.br%2Fesaj%2Fj_spring_cas_security_check')
    driver.find_element_by_xpath('//*[@id="usernameForm"]').send_keys('01378744055')
    driver.find_element_by_xpath('//*[@id="passwordForm"]').send_keys('rasengan')
    driver.find_element_by_xpath('//*[@id="pbEntrar"]').click()
    driver.get('https://esaj.tjsp.jus.br/cposg/open.do')

    vazio = 0
    pacote = 1000

    if SegundaInstancia.objects.all().exists():
        if progressivo:
            q = SegundaInstancia.objects.all().order_by('-seq').only('seq')[:pacote]
            q = [i.seq for i in q]
            SEQUENCIAS = [i for i in range(min(q), min(q) + (pacote*2)) if i not in q]
            ano = SegundaInstancia.objects.all().order_by('-ano').first().ano


    else:
        seq = 2262000

    for seq in SEQUENCIAS:

        params = {

        'conversationId': '',
        'paginaConsulta': '1',
        'localPesquisa.cdLocal': '-1',
        'cbPesquisa': 'NUMPROC',
        'tipoNuProcesso': 'UNIFICADO',
        'dePesquisaNuUnificado': calculo_cnj(seq, ano, 8, 26, 0),
        'dePesquisa': '',
        'uuidCaptcha': '',
        }

        r = driver.request('GET', 'https://esaj.tjsp.jus.br/cposg/search.do', params=params)

        res = {'seq': seq,
               'ano': ano,
               'info_url' : r.url}

        sp = bs(r.content, 'html5lib')

        if sp.find(attrs={'role':'alert'}) and not sp.find(text={re.compile('Será necessário informar uma senha para acessar processos em segredo de justiça')}):
            vazio +=1

        else:
            vazio = 0
            if sp.find(text={re.compile('Será necessário informar uma senha para acessar processos em segredo de justiça')}):
                res['segredo_de_justica'] = True
            else:
                tabela_dados_gerais = sp.findAll(attrs={'class':'secaoFormBody'})[1].find('tbody')
                for entrada in tabela_dados_gerais.findAll('tr'):
                    try:
                        k = entrada.find('label').get_text()
                        v = entrada.find('span').get_text()
                        res[k] = v
                    except:
                        pass

                tabela_1_instancia = sp.find(text=re.compile('Números de 1ª Instância')).findAllNext('table')[1].find('tbody')
                if tabela_1_instancia:
                    coleta = []
                    for entrada in tabela_1_instancia.findAll('tr'):
                        coleta.append({k: v.get_text().strip() for k,v in zip(['n_primeira_instancia', 'foro', 'vara', 'juiz', 'obs'], entrada.findAll('td')) })    
                    if len(coleta) > 0:
                        res['numeros_1_instancia'] = coleta

                tabela_partes = sp.find(text=re.compile('Partes do Processo')).findNext('table').find('tbody')

                coleta = []
                for entrada in tabela_partes.findAll('tr'):
                    coleta.append({'posicao' : entrada.findAll('td')[0].find('span').get_text(),
                                   'nome': entrada.findAll('td')[1].next.strip()})
                if len(coleta) > 0:
                    res['partes'] = coleta
                if sp.find(attrs={'id':'tabelaTodasMovimentacoes'}):
                    coleta = []
                    for _, entrada in enumerate(sp.find(attrs={'id':'tabelaTodasMovimentacoes'}).findAll('tr')[::-1]):
                        linhas = entrada.findAll('td')
                        rp = {'ordem': _}
                        rp['data'] = datetime.datetime.strptime(linhas[0].get_text().strip(), '%d/%m/%Y').date()

                        if linhas[1].find('a'):
                            rp['link0'] = linhas[1].find('a').get('href')

                        if linhas[2].find('a'):
                            rp['link1'] = linhas[2].find('a').get('href')
                            rp['movimentacaotag'] =  linhas[2].find('a').get_text().strip()
                        else:
                            rp['movimentacaotag'] = linhas[2].next.strip()

                        rp['movimentacaotag'], x = MovimentacaoTag.objects.get_or_create(nome=rp['movimentacaotag'])

                        rp['movimentacaodescricao'] = linhas[2].find('span').get_text()
                        coleta.append(rp)
                    if len(coleta)>0:
                        res['movimentacoes'] = coleta

            res = {k.replace(':','').lower().strip(): v for k,v in res.items()}
            res = {unidecode(k).replace(' ','_').replace('_/_','_'): (v.strip() if isinstance(v, str) else v) for k,v in res.items()}

            if 'valor_da_acao' in res.keys():
                res['valor_da_acao'] = float(res['valor_da_acao'].replace('.','').replace(',','.'))

            SegundaInstancia_m = SegundaInstancia(**{k:v for k,v in res.items() if k not in ['partes','numeros_1_instancia', 'movimentacoes' ]})
            SegundaInstancia_m.save()

            if 'partes' in res.keys():
                partes = [{k: (v.lower().strip().replace(':','') if k =='posicao' else v) for k,v in i.items()} for i in res['partes']]

                for p in partes:
                    p = Parte.objects.get_or_create(nome = p['nome']), p['posicao']
                    posicao = Posicao(segunda_instancia=SegundaInstancia_m, parte=p[0][0], posicao=p[1])
                    posicao.save()

            if 'numeros_1_instancia' in res.keys():
                n_1_instancia = [{k:v for k,v in i.items() if v !='-'} for i in res['numeros_1_instancia']]
                Numero1Instancia.objects.bulk_create([Numero1Instancia(**i, segunda_instancia=SegundaInstancia_m) for i in n_1_instancia])

            if 'movimentacoes' in res.keys():
                m = [Movimentacao(**i, segunda_instancia=SegundaInstancia_m) for i in res['movimentacoes']]
                Movimentacao.objects.bulk_create(m)

        if vazio == int(pacote*0.5):
            break

        print(seq, vazio)
        
scrapper_2_instancia()

2267316 0
2267321 0
2267322 0
2267323 0
2267324 0
2267325 0
2267326 0
2267327 0
2267328 0
2267329 0
2267330 0
2267331 0
2267332 0
2267333 0
2267334 0
2267335 0
2267336 0
2267337 0
2267338 0
2267339 0
2267340 0
2267341 0
2267342 0
2267343 0
2267344 0
2267345 0
2267346 0
2267347 0
2267348 0
2267349 0
2267350 0
2267351 0
2267352 0
2267353 0
2267354 0
2267355 0
2267356 0
2267357 0
2267358 0
2267359 0
2267360 0
2267361 0
2267362 0
2267363 0
2267364 0
2267365 0
2267366 0
2267367 0
2267368 0
2267369 0
2267370 0
2267371 0
2267372 0
2267373 0
2267374 0
2267375 0
2267376 1
2267377 2
2267378 3
2267379 4
2267380 5
2267381 6
2267382 7
2267383 8
2267384 9
2267385 10
2267386 11
2267387 12
2267388 13
2267389 14
2267390 15
2267391 16
2267392 17
2267393 18
2267394 19
2267395 20
2267396 21
2267397 22
2267398 23
2267399 24
2267400 25
2267401 26
2267402 27
2267403 28
2267404 29
2267405 30
2267406 31
2267407 32
2267408 33
2267409 34
2267410 35
2267411 36
2267412 37
2267413 38
2267414 39
2267415 40
2267416 4

In [2]:
from bs4 import BeautifulSoup as bs
from seleniumrequests import Chrome
import re
from unidecode import unidecode
from core.models import *
from django import db
import datetime


q = SegundaInstancia.objects.earliest('id').id
q
# q = [i.seq for i in q]
# SEQUENCIAS = [i for i in range(min(q), min(q) + (pacote*2)) if i not in q]
# ano = SegundaInstancia.objects.all().order_by('-ano').first().ano


15

In [3]:
from bs4 import BeautifulSoup as bs
from seleniumrequests import Chrome
import re
from unidecode import unidecode
from core.models import *
from django import db
import datetime


SegundaInstancia.objects.all().order_by('-ano').first().ano

2019

In [22]:
from bs4 import BeautifulSoup as bs
from seleniumrequests import Chrome
import re
from unidecode import unidecode
from core.models import *
from django import db
import datetime

db.close_old_connections()

def scrapper_2_instancia(sentido='progressivo'):
    
    def calculo_cnj(seq, ano, j, tr, vara):
        vara = str(vara)
        while len(vara) < 4:
            vara = vara + '0'
        n = int(f'{seq}{ano}{j}{tr}{vara}00')
        d = str(98 - n%97)
        if len(d) == 1:
            d = '0' + d
        seq = str(seq)
        while len(seq) < 7:
            seq = '0' + seq
        return f'{seq}-{d}.{ano}.{j}.{tr}.{vara}'


    driver = Chrome()

    driver.get('https://esaj.tjsp.jus.br/sajcas/login?service=https%3A%2F%2Fesaj.tjsp.jus.br%2Fesaj%2Fj_spring_cas_security_check')
    driver.find_element_by_xpath('//*[@id="usernameForm"]').send_keys('01378744055')
    driver.find_element_by_xpath('//*[@id="passwordForm"]').send_keys('rasengan')
    driver.find_element_by_xpath('//*[@id="pbEntrar"]').click()
    driver.get('https://esaj.tjsp.jus.br/cposg/open.do')

    vazio = 0

    if SegundaInstancia.objects.all().exists():
        seq = SegundaInstancia.objects.latest('seq').seq + 1
    else:
        seq = 2262000

    while True:

        params = {

        'conversationId': '',
        'paginaConsulta': '1',
        'localPesquisa.cdLocal': '-1',
        'cbPesquisa': 'NUMPROC',
        'tipoNuProcesso': 'UNIFICADO',
        'dePesquisaNuUnificado': calculo_cnj(seq, 2019, 8, 26, 0),
        'dePesquisa': '',
        'uuidCaptcha': '',
        }

        r = driver.request('GET', 'https://esaj.tjsp.jus.br/cposg/search.do', params=params)

        res = {'seq': seq,
               'info_url' : r.url}

        sp = bs(r.content, 'html5lib')

        if sp.find(attrs={'role':'alert'}) and not sp.find(text={re.compile('Será necessário informar uma senha para acessar processos em segredo de justiça')}):
            vazio +=1

        else:
            vazio = 0
            if sp.find(text={re.compile('Será necessário informar uma senha para acessar processos em segredo de justiça')}):
                res['segredo_de_justica'] = True
            else:
                tabela_dados_gerais = sp.findAll(attrs={'class':'secaoFormBody'})[1].find('tbody')
                for entrada in tabela_dados_gerais.findAll('tr'):
                    try:
                        k = entrada.find('label').get_text()
                        v = entrada.find('span').get_text()
                        res[k] = v
                    except:
                        pass

                tabela_1_instancia = sp.find(text=re.compile('Números de 1ª Instância')).findAllNext('table')[1].find('tbody')
                if tabela_1_instancia:
                    coleta = []
                    for entrada in tabela_1_instancia.findAll('tr'):
                        coleta.append({k: v.get_text().strip() for k,v in zip(['n_primeira_instancia', 'foro', 'vara', 'juiz', 'obs'], entrada.findAll('td')) })    
                    if len(coleta) > 0:
                        res['numeros_1_instancia'] = coleta

                tabela_partes = sp.find(text=re.compile('Partes do Processo')).findNext('table').find('tbody')

                coleta = []
                for entrada in tabela_partes.findAll('tr'):
                    coleta.append({'posicao' : entrada.findAll('td')[0].find('span').get_text(),
                                   'nome': entrada.findAll('td')[1].next.strip()})
                if len(coleta) > 0:
                    res['partes'] = coleta
                    
                if sp.find(attrs={'id':'tabelaTodasMovimentacoes'}):
                    coleta = []
                    for _, entrada in enumerate(sp.find(attrs={'id':'tabelaTodasMovimentacoes'}).findAll('tr')[::-1]):
                        linhas = entrada.findAll('td')
                        rp = {'ordem': _}
                        rp['data'] = datetime.datetime.strptime(linhas[0].get_text().strip(), '%d/%m/%Y').date()

                        if linhas[1].find('a'):
                            rp['link0'] = linhas[1].find('a').get('href')

                        if linhas[2].find('a'):
                            rp['link1'] = linhas[2].find('a').get('href')
                            rp['movimentacaotag'] =  linhas[2].find('a').get_text().strip()
                        else:
                            rp['movimentacaotag'] = linhas[2].next.strip()

                        rp['movimentacaotag'], x = MovimentacaoTag.objects.get_or_create(nome=rp['movimentacaotag'])

                        rp['movimentacaodescricao'] = linhas[2].find('span').get_text()
                        coleta.append(rp)
                    if len(coleta)>0:
                        res['movimentacoes'] = coleta

            res = {k.replace(':','').lower().strip(): v for k,v in res.items()}
            res = {unidecode(k).replace(' ','_').replace('_/_','_'): (v.strip() if isinstance(v, str) else v) for k,v in res.items()}

            if 'valor_da_acao' in res.keys():
                res['valor_da_acao'] = float(res['valor_da_acao'].replace('.','').replace(',','.'))

            SegundaInstancia_m = SegundaInstancia(**{k:v for k,v in res.items() if k not in ['partes','numeros_1_instancia', 'movimentacoes' ]})
            SegundaInstancia_m.save()

            if 'partes' in res.keys():
                partes = [{k: (v.lower().strip().replace(':','') if k =='posicao' else v) for k,v in i.items()} for i in res['partes']]

                for p in partes:
                    p = Parte.objects.get_or_create(nome = p['nome']), p['posicao']
                    posicao = Posicao(segunda_instancia=SegundaInstancia_m, parte=p[0][0], posicao=p[1])
                    posicao.save()

            if 'numeros_1_instancia' in res.keys():
                n_1_instancia = [{k:v for k,v in i.items() if v !='-'} for i in res['numeros_1_instancia']]
                Numero1Instancia.objects.bulk_create([Numero1Instancia(**i, segunda_instancia=SegundaInstancia_m) for i in n_1_instancia])

            if 'movimentacoes' in res.keys():
                m = [Movimentacao(**i, segunda_instancia=SegundaInstancia_m) for i in res['movimentacoes']]
                Movimentacao.objects.bulk_create(m)

        seq += 1
        if vazio == 25:
            break

        print(seq, vazio)
#         print(res['classe'])

scrapper_2_instancia()
# while True:
#     for sent in ['regressivo', 'progressivo']:
#         scrapper_2_instancia(sentido=sent)


2266242 0
2266243 0
2266244 0
2266245 0
2266246 0
2266247 0
2266248 0
2266249 0
2266250 0
2266251 0
2266252 0
2266253 0
2266254 0
2266255 0
2266256 0
2266257 0
2266258 0
2266259 0
2266260 0
2266261 0
2266262 0
2266263 0
2266264 0
2266265 0
2266266 0
2266267 0
2266268 0
2266269 0
2266270 0
2266271 0
2266272 0
2266273 0
2266274 0
2266275 0
2266276 0
2266277 0
2266278 0
2266279 0
2266280 0
2266281 0
2266282 0
2266283 0
2266284 0
2266285 0
2266286 0
2266287 0
2266288 0
2266289 0
2266290 0
2266291 0
2266292 0
2266293 0
2266294 0
2266295 0
2266296 0
2266297 0
2266298 0
2266299 0
2266300 0
2266301 0
2266302 0
2266303 0
2266304 0
2266305 0
2266306 0
2266307 0
2266308 0
2266309 0
2266310 0
2266311 0
2266312 0
2266313 0
2266314 0
2266315 0
2266316 0
2266317 0
2266318 0
2266319 0
2266320 0
2266321 0
2266322 0
2266323 0
2266324 0
2266325 0
2266326 0
2266327 0
2266328 0
2266329 0
2266330 0
2266331 0
2266332 0
2266333 0
2266334 0
2266335 0
2266336 0
2266337 0
2266338 0
2266339 0
2266340 0
2266341 0


KeyboardInterrupt: 

In [3]:


print(x)

      |\      _,,,---,,_
ZZZzz /,`.-'`'    -.  ;-;;,_
     |,4-  ) )-,_. ,\ (  `'-'
    '---''(_/--'  `-'\_)


In [13]:
from bs4 import BeautifulSoup as bs
from seleniumrequests import Chrome
import re
from unidecode import unidecode
from core.models import *
from django import db
import datetime

db.close_old_connections()

def scrapper_2_instancia(sentido='progressivo'):
    
    def calculo_cnj(seq, ano, j, tr, vara):
        vara = str(vara)
        while len(vara) < 4:
            vara = vara + '0'
        n = int(f'{seq}{ano}{j}{tr}{vara}00')
        d raz
        = str(98 - n%97)
        if len(d) == 1:
            d = '0' + d
        seq = str(seq)
        while len(seq) < 7:
            seq = '0' + seq
        return f'{seq}-{d}.{ano}.{j}.{tr}.{vara}'


    driver = Chrome()

    driver.get('https://esaj.tjsp.jus.br/sajcas/login?service=https%3A%2F%2Fesaj.tjsp.jus.br%2Fesaj%2Fj_spring_cas_security_check')
    driver.find_element_by_xpath('//*[@id="usernameForm"]').send_keys('01378744055')
    driver.find_element_by_xpath('//*[@id="passwordForm"]').send_keys('rasengan')
    driver.find_element_by_xpath('//*[@id="pbEntrar"]').click()
    driver.get('https://esaj.tjsp.jus.br/cposg/open.do')

    vazio = 0

    if SegundaInstancia.objects.all().exists():
        seq = SegundaInstancia.objects.earliest('seq').seq - 1
    else:
        seq = 2262000

    while True:

        params = {

        'conversationId': '',
        'paginaConsulta': '1',
        'localPesquisa.cdLocal': '-1',
        'cbPesquisa': 'NUMPROC',
        'tipoNuProcesso': 'UNIFICADO',
        'dePesquisaNuUnificado': calculo_cnj(seq, 2019, 8, 26, 0),
        'dePesquisa': '',
        'uuidCaptcha': '',
        }

        r = driver.request('GET', 'https://esaj.tjsp.jus.br/cposg/search.do', params=params)

        res = {'seq': seq,
               'info_url' : r.url}

        sp = bs(r.content, 'html5lib')

        if sp.find(attrs={'role':'alert'}) and not sp.find(text={re.compile('Será necessário informar uma senha para acessar processos em segredo de justiça')}):
            vazio +=1

        else:
            vazio = 0
            if sp.find(text={re.compile('Será necessário informar uma senha para acessar processos em segredo de justiça')}):
                res['segredo_de_justica'] = True
            else:
                tabela_dados_gerais = sp.findAll(attrs={'class':'secaoFormBody'})[1].find('tbody')
                for entrada in tabela_dados_gerais.findAll('tr'):
                    try:
                        k = entrada.find('label').get_text()
                        v = entrada.find('span').get_text()
                        res[k] = v
                    except:
                        pass

                tabela_1_instancia = sp.find(text=re.compile('Números de 1ª Instância')).findAllNext('table')[1].find('tbody')
                if tabela_1_instancia:
                    coleta = []
                    for entrada in tabela_1_instancia.findAll('tr'):
                        coleta.append({k: v.get_text().strip() for k,v in zip(['n_primeira_instancia', 'foro', 'vara', 'juiz', 'obs'], entrada.findAll('td')) })    
                    if len(coleta) > 0:
                        res['numeros_1_instancia'] = coleta

                tabela_partes = sp.find(text=re.compile('Partes do Processo')).findNext('table').find('tbody')

                coleta = []
                for entrada in tabela_partes.findAll('tr'):
                    coleta.append({'posicao' : entrada.findAll('td')[0].find('span').get_text(),
                                   'nome': entrada.findAll('td')[1].next.strip()})
                if len(coleta) > 0:
                    res['partes'] = coleta
                    
                if sp.find(attrs={'id':'tabelaTodasMovimentacoes'}):
                    coleta = []
                    for _, entrada in enumerate(sp.find(attrs={'id':'tabelaTodasMovimentacoes'}).findAll('tr')[::-1]):
                        linhas = entrada.findAll('td')
                        rp = {'ordem': _}
                        rp['data'] = datetime.datetime.strptime(linhas[0].get_text().strip(), '%d/%m/%Y').date()

                        if linhas[1].find('a'):
                            rp['link0'] = linhas[1].find('a').get('href')

                        if linhas[2].find('a'):
                            rp['link1'] = linhas[2].find('a').get('href')
                            rp['movimentacaotag'] =  linhas[2].find('a').get_text().strip()
                        else:
                            rp['movimentacaotag'] = linhas[2].next.strip()

                        rp['movimentacaotag'], x = MovimentacaoTag.objects.get_or_create(nome=rp['movimentacaotag'])

                        rp['movimentacaodescricao'] = linhas[2].find('span').get_text()
                        coleta.append(rp)
                    if len(coleta)>0:
                        res['movimentacoes'] = coleta

            res = {k.replace(':','').lower().strip(): v for k,v in res.items()}
            res = {unidecode(k).replace(' ','_').replace('_/_','_'): (v.strip() if isinstance(v, str) else v) for k,v in res.items()}

            if 'valor_da_acao' in res.keys():
                res['valor_da_acao'] = float(res['valor_da_acao'].replace('.','').replace(',','.'))

            SegundaInstancia_m = SegundaInstancia(**{k:v for k,v in res.items() if k not in ['partes','numeros_1_instancia', 'movimentacoes' ]})
            SegundaInstancia_m.save()

            if 'partes' in res.keys():
                partes = [{k: (v.lower().strip().replace(':','') if k =='posicao' else v) for k,v in i.items()} for i in res['partes']]

                for p in partes:
                    p = Parte.objects.get_or_create(nome = p['nome']), p['posicao']
                    posicao = Posicao(segunda_instancia=SegundaInstancia_m, parte=p[0][0], posicao=p[1])
                    posicao.save()

            if 'numeros_1_instancia' in res.keys():
                n_1_instancia = [{k:v for k,v in i.items() if v !='-'} for i in res['numeros_1_instancia']]
                Numero1Instancia.objects.bulk_create([Numero1Instancia(**i, segunda_instancia=SegundaInstancia_m) for i in n_1_instancia])

            if 'movimentacoes' in res.keys():
                m = [Movimentacao(**i, segunda_instancia=SegundaInstancia_m) for i in res['movimentacoes']]
                Movimentacao.objects.bulk_create(m)

        seq -= 1
        if vazio == 25:
            break

        print(seq, vazio)

scrapper_2_instancia()
# while True:
#     for sent in ['regressivo', 'progressivo']:
#         scrapper_2_instancia(sentido=sent)


2259743 0
2259742 0
2259741 0
2259740 0
2259739 0
2259738 0
2259737 0
2259736 0
2259735 0
2259734 0
2259733 0
2259732 0
2259731 0
2259730 0
2259729 0
2259728 0
2259727 0
2259726 0
2259725 0
2259724 0
2259723 0
2259722 0
2259721 0
2259720 0
2259719 0
2259718 0
2259717 0
2259716 0
2259715 0
2259714 0
2259713 0
2259712 0
2259711 0
2259710 0
2259709 0
2259708 0
2259707 0
2259706 0
2259705 0
2259704 0
2259703 0
2259702 0
2259701 0
2259700 0
2259699 0
2259698 0
2259697 0
2259696 0
2259695 0
2259694 0
2259693 0
2259692 0
2259691 0
2259690 0
2259689 0
2259688 0
2259687 0
2259686 0
2259685 0
2259684 0
2259683 0
2259682 0
2259681 0
2259680 0
2259679 0
2259678 0
2259677 0
2259676 0
2259675 0
2259674 0
2259673 0
2259672 0
2259671 0
2259670 0
2259669 0
2259668 0
2259667 0
2259666 0
2259665 0
2259664 0
2259663 0
2259662 0
2259661 0
2259660 0
2259659 0
2259658 0
2259657 0
2259656 0
2259655 0
2259654 0
2259653 0
2259652 0
2259651 0
2259650 0
2259649 0
2259648 0
2259647 0
2259646 0
2259645 0
2259644 0


2258923 0
2258922 0
2258921 0
2258920 0
2258919 0
2258918 0
2258917 0
2258916 0
2258915 0
2258914 0
2258913 0
2258912 0
2258911 0
2258910 0
2258909 0
2258908 0
2258907 0
2258906 0
2258905 0
2258904 0
2258903 0
2258902 0
2258901 0
2258900 0
2258899 0
2258898 0
2258897 0
2258896 0
2258895 0
2258894 0
2258893 0
2258892 0
2258891 0
2258890 0
2258889 0
2258888 0
2258887 0
2258886 0
2258885 0
2258884 0
2258883 0
2258882 0
2258881 0
2258880 0
2258879 0
2258878 0
2258877 0
2258876 0
2258875 0
2258874 0
2258873 0
2258872 0
2258871 0
2258870 0
2258869 0
2258868 0
2258867 0
2258866 0
2258865 0
2258864 0
2258863 0
2258862 0
2258861 0
2258860 0
2258859 0
2258858 0
2258857 0
2258856 0
2258855 0
2258854 0
2258853 0
2258852 0
2258851 0
2258850 0
2258849 0
2258848 0
2258847 0
2258846 0
2258845 0
2258844 0
2258843 0
2258842 0
2258841 0
2258840 0
2258839 0
2258838 0
2258837 0
2258836 0
2258835 0
2258834 0
2258833 0
2258832 0
2258831 0
2258830 0
2258829 0
2258828 1
2258827 0
2258826 1
2258825 2
2258824 0


2258103 0
2258102 0
2258101 0
2258100 0
2258099 0
2258098 0
2258097 0
2258096 0
2258095 0
2258094 0
2258093 0
2258092 0
2258091 0
2258090 0
2258089 0
2258088 0
2258087 0
2258086 0
2258085 0
2258084 0
2258083 0
2258082 0
2258081 0
2258080 0
2258079 0
2258078 0
2258077 0
2258076 0
2258075 0
2258074 0
2258073 0
2258072 0
2258071 0
2258070 0
2258069 0
2258068 0
2258067 0
2258066 0
2258065 0
2258064 0
2258063 0
2258062 0
2258061 0
2258060 0
2258059 0
2258058 0
2258057 0
2258056 0
2258055 0
2258054 0
2258053 0
2258052 0
2258051 0
2258050 0
2258049 0
2258048 0
2258047 0
2258046 0
2258045 0
2258044 0
2258043 0
2258042 0
2258041 0
2258040 0
2258039 0
2258038 0
2258037 0
2258036 0
2258035 0
2258034 0
2258033 0
2258032 0
2258031 0
2258030 0
2258029 0
2258028 0
2258027 0
2258026 0
2258025 0
2258024 0
2258023 0
2258022 0
2258021 0
2258020 0
2258019 0
2258018 0
2258017 0
2258016 0
2258015 0
2258014 0
2258013 0
2258012 0
2258011 0
2258010 0
2258009 0
2258008 0
2258007 0
2258006 0
2258005 0
2258004 0


2257283 0
2257282 0
2257281 0
2257280 0
2257279 0
2257278 0
2257277 0
2257276 0
2257275 0
2257274 0
2257273 0
2257272 0
2257271 0
2257270 0
2257269 0
2257268 0
2257267 0
2257266 0
2257265 0
2257264 0
2257263 0
2257262 0
2257261 0
2257260 0
2257259 0
2257258 0
2257257 0
2257256 0
2257255 0
2257254 0
2257253 0
2257252 0
2257251 0
2257250 0
2257249 0
2257248 0
2257247 0
2257246 0
2257245 0
2257244 0
2257243 0
2257242 0
2257241 0
2257240 0
2257239 0
2257238 0
2257237 0
2257236 0
2257235 0
2257234 0
2257233 0
2257232 0
2257231 0
2257230 0
2257229 0
2257228 0
2257227 0
2257226 0
2257225 0
2257224 0
2257223 0
2257222 0
2257221 0
2257220 0
2257219 0
2257218 0
2257217 0
2257216 0
2257215 0
2257214 0
2257213 0
2257212 0
2257211 0
2257210 0
2257209 0
2257208 0
2257207 0
2257206 0
2257205 0
2257204 0
2257203 0
2257202 0
2257201 0
2257200 0
2257199 0
2257198 0
2257197 0
2257196 0
2257195 0
2257194 0
2257193 0
2257192 0
2257191 0
2257190 0
2257189 0
2257188 0
2257187 0
2257186 0
2257185 0
2257184 0


2256463 0
2256462 0
2256461 0
2256460 0
2256459 0
2256458 0
2256457 0
2256456 0
2256455 0
2256454 0
2256453 0
2256452 0
2256451 0
2256450 0
2256449 0
2256448 0
2256447 0
2256446 0
2256445 0
2256444 0
2256443 0
2256442 0
2256441 0
2256440 0
2256439 0
2256438 0
2256437 0
2256436 0
2256435 0
2256434 0
2256433 0
2256432 0
2256431 0
2256430 0
2256429 0
2256428 0
2256427 0
2256426 0
2256425 0
2256424 0
2256423 0
2256422 0
2256421 0
2256420 0
2256419 0
2256418 0
2256417 0
2256416 0
2256415 0
2256414 0
2256413 0
2256412 0
2256411 0
2256410 0
2256409 0
2256408 0
2256407 0
2256406 0
2256405 0
2256404 0
2256403 0
2256402 0
2256401 0
2256400 0
2256399 0
2256398 0
2256397 0
2256396 0
2256395 0
2256394 0
2256393 0
2256392 0
2256391 0
2256390 0
2256389 0
2256388 0
2256387 0
2256386 0
2256385 0
2256384 0
2256383 0
2256382 0
2256381 0
2256380 0
2256379 0
2256378 0
2256377 0
2256376 0
2256375 0
2256374 0
2256373 0
2256372 0
2256371 0
2256370 0
2256369 0
2256368 0
2256367 0
2256366 0
2256365 0
2256364 0


...

2253183 0
2253182 0
2253181 0
2253180 0
2253179 0
2253178 0
2253177 0
2253176 0
2253175 0
2253174 0
2253173 0
2253172 0
2253171 0
2253170 0
2253169 0
2253168 0
2253167 0
2253166 0
2253165 0
2253164 0
2253163 0
2253162 0
2253161 0
2253160 0
2253159 0
2253158 0
2253157 0
2253156 0
2253155 0
2253154 0
2253153 0
2253152 0
2253151 0
2253150 0
2253149 0
2253148 0
2253147 0
2253146 0
2253145 0
2253144 0
2253143 0
2253142 0
2253141 0
2253140 0
2253139 0
2253138 0
2253137 0
2253136 0
2253135 0
2253134 0
2253133 0
2253132 0
2253131 0
2253130 0
2253129 0
2253128 0
2253127 0
2253126 0
2253125 0
2253124 0
2253123 0
2253122 0
2253121 0
2253120 0
2253119 0
2253118 0
2253117 0
2253116 0
2253115 0
2253114 0
2253113 0
2253112 0
2253111 0
2253110 0
2253109 0
2253108 0
2253107 0
2253106 0
2253105 0
2253104 0
2253103 0
2253102 0
2253101 0
2253100 0
2253099 0
2253098 0
2253097 0
2253096 0
2253095 0
2253094 0
2253093 0
2253092 0
2253091 0
2253090 0
2253089 0
2253088 0
2253087 0
2253086 0
2253085 0
2253084 0


2252363 0
2252362 0
2252361 0
2252360 0
2252359 0
2252358 0
2252357 0
2252356 0
2252355 0
2252354 0
2252353 0
2252352 0
2252351 0
2252350 0
2252349 0
2252348 0
2252347 0
2252346 0
2252345 0
2252344 0
2252343 0
2252342 0
2252341 0
2252340 0
2252339 0
2252338 0
2252337 0
2252336 0
2252335 0
2252334 0
2252333 0
2252332 0
2252331 0
2252330 0
2252329 0
2252328 0
2252327 0
2252326 0
2252325 0
2252324 0
2252323 0
2252322 0
2252321 0
2252320 0
2252319 0
2252318 0
2252317 0
2252316 0
2252315 0
2252314 0
2252313 0
2252312 0
2252311 0
2252310 0
2252309 0
2252308 0
2252307 0
2252306 0
2252305 0
2252304 0
2252303 0
2252302 0
2252301 0
2252300 0
2252299 0
2252298 0
2252297 0
2252296 0
2252295 0
2252294 0
2252293 0
2252292 0
2252291 0
2252290 0
2252289 0
2252288 0
2252287 0
2252286 0
2252285 0
2252284 0
2252283 0
2252282 0
2252281 0
2252280 0
2252279 0
2252278 0
2252277 0
2252276 0
2252275 0
2252274 0
2252273 0
2252272 0
2252271 0
2252270 0
2252269 0
2252268 0
2252267 0
2252266 0
2252265 0
2252264 0


2251543 0
2251542 0
2251541 0
2251540 0
2251539 0
2251538 0
2251537 0
2251536 0
2251535 0
2251534 0
2251533 0
2251532 0
2251531 0
2251530 0
2251529 0
2251528 0
2251527 0
2251526 0
2251525 0
2251524 0
2251523 0
2251522 0
2251521 0
2251520 0
2251519 0
2251518 0
2251517 0
2251516 0
2251515 0
2251514 0
2251513 0
2251512 0
2251511 0
2251510 0
2251509 0
2251508 0
2251507 0
2251506 0
2251505 0
2251504 0
2251503 0
2251502 0
2251501 0
2251500 0
2251499 0
2251498 0
2251497 0
2251496 0
2251495 0
2251494 0
2251493 0
2251492 0
2251491 0
2251490 0
2251489 0
2251488 0
2251487 0
2251486 0
2251485 0
2251484 1
2251483 0
2251482 0
2251481 0
2251480 0
2251479 0
2251478 0
2251477 0
2251476 0
2251475 1
2251474 0
2251473 0
2251472 0
2251471 0
2251470 0
2251469 0
2251468 0
2251467 0
2251466 0
2251465 0
2251464 0
2251463 0
2251462 0
2251461 0
2251460 0
2251459 0
2251458 0
2251457 0
2251456 0
2251455 0
2251454 0
2251453 0
2251452 0
2251451 0
2251450 0
2251449 0
2251448 0
2251447 0
2251446 0
2251445 0
2251444 0


2250723 0
2250722 0
2250721 0
2250720 0
2250719 0
2250718 0
2250717 0
2250716 0
2250715 0
2250714 0
2250713 0
2250712 0
2250711 0
2250710 0
2250709 0
2250708 0
2250707 0
2250706 0
2250705 0
2250704 0
2250703 0
2250702 0
2250701 0
2250700 0
2250699 0
2250698 0
2250697 0
2250696 0
2250695 0
2250694 0
2250693 0
2250692 0
2250691 0
2250690 0
2250689 0
2250688 0
2250687 0
2250686 0
2250685 0
2250684 0
2250683 0
2250682 0
2250681 0
2250680 0
2250679 0
2250678 0
2250677 0
2250676 0
2250675 0
2250674 0
2250673 0
2250672 0
2250671 0
2250670 0
2250669 0
2250668 0
2250667 0
2250666 0
2250665 0
2250664 0
2250663 0
2250662 0
2250661 0
2250660 0
2250659 0
2250658 0
2250657 0
2250656 0
2250655 0
2250654 0
2250653 0
2250652 0
2250651 0
2250650 0
2250649 0
2250648 0
2250647 0
2250646 0
2250645 0
2250644 0
2250643 0
2250642 0
2250641 0
2250640 0
2250639 0
2250638 0
2250637 0
2250636 0
2250635 0
2250634 0
2250633 0
2250632 0
2250631 0
2250630 0
2250629 0
2250628 0
2250627 0
2250626 0
2250625 0
2250624 0


TypeError: SegundaInstancia() got an unexpected keyword argument 'revisor'

In [7]:
def calculo_cnj(seq, ano, j, tr, vara):
    vara = str(vara)
    while len(vara) < 4:
        vara = vara + '0'
    n = int(f'{seq}{ano}{j}{tr}{vara}00')
    d = str(98 - n%97)
    if len(d) == 1:
        d = '0' + d
    seq = str(seq)
    while len(seq) < 7:
        seq = '0' + seq
    return f'{seq}-{d}.{ano}.{j}.{tr}.{vara}'

calculo_cnj(2264743, 2019, 8, 26, 0)

'2264743-40.2019.8.26.0000'

In [11]:
from bs4 import BeautifulSoup as bs
from seleniumrequests import Chrome
import re
from unidecode import unidecode
from core.models import *
from django import db
import datetime

db.close_old_connections()

def calculo_cnj(seq, ano, j, tr, vara):
    vara = str(vara)
    while len(vara) < 4:
        vara = vara + '0'
    n = int(f'{seq}{ano}{j}{tr}{vara}00')
    d = str(98 - n%97)
    if len(d) == 1:
        d = '0' + d
    seq = str(seq)
    while len(seq) < 7:
        seq = '0' + seq
    return f'{seq}-{d}.{ano}.{j}.{tr}.{vara}'


driver = Chrome()

driver.get('https://esaj.tjsp.jus.br/sajcas/login?service=https%3A%2F%2Fesaj.tjsp.jus.br%2Fesaj%2Fj_spring_cas_security_check')
driver.find_element_by_xpath('//*[@id="usernameForm"]').send_keys('01378744055')
driver.find_element_by_xpath('//*[@id="passwordForm"]').send_keys('rasengan')
driver.find_element_by_xpath('//*[@id="pbEntrar"]').click()
driver.get('https://esaj.tjsp.jus.br/cposg/open.do')

vazio = 0

if SegundaInstancia.objects.all().exists():
    seq = SegundaInstancia.objects.latest('seq').seq + 1
else:
    seq = 2262000

while True:

    params = {

    'conversationId': '',
    'paginaConsulta': '1',
    'localPesquisa.cdLocal': '-1',
    'cbPesquisa': 'NUMPROC',
    'tipoNuProcesso': 'UNIFICADO',
    'dePesquisaNuUnificado': calculo_cnj(seq, 2019, 8, 26, 0),
    'dePesquisa': '',
    'uuidCaptcha': '',
    }

    r = driver.request('GET', 'https://esaj.tjsp.jus.br/cposg/search.do', params=params)

    res = {'seq': seq,
           'info_url' : r.url}

    sp = bs(r.content, 'html5lib')

    if sp.find(attrs={'role':'alert'}) and not sp.find(text={re.compile('Será necessário informar uma senha para acessar processos em segredo de justiça')}):
        vazio +=1

    else:
        vazio = 0
        if sp.find(text={re.compile('Será necessário informar uma senha para acessar processos em segredo de justiça')}):
            res['segredo_de_justica'] = True
        else:
            tabela_dados_gerais = sp.findAll(attrs={'class':'secaoFormBody'})[1].find('tbody')
            for entrada in tabela_dados_gerais.findAll('tr'):
                try:
                    k = entrada.find('label').get_text()
                    v = entrada.find('span').get_text()
                    res[k] = v
                except:
                    pass

            tabela_1_instancia = sp.find(text=re.compile('Números de 1ª Instância')).findAllNext('table')[1].find('tbody')
            if tabela_1_instancia:
                coleta = []
                for entrada in tabela_1_instancia.findAll('tr'):
                    coleta.append({k: v.get_text().strip() for k,v in zip(['n_primeira_instancia', 'foro', 'vara', 'juiz', 'obs'], entrada.findAll('td')) })    
                if len(coleta) > 0:
                    res['numeros_1_instancia'] = coleta

            tabela_partes = sp.find(text=re.compile('Partes do Processo')).findNext('table').find('tbody')

            coleta = []
            for entrada in tabela_partes.findAll('tr'):
                coleta.append({'posicao' : entrada.findAll('td')[0].find('span').get_text(),
                               'nome': entrada.findAll('td')[1].next.strip()})
            if len(coleta) > 0:
                res['partes'] = coleta
                
        coleta = []
        for _, entrada in enumerate(sp.find(attrs={'id':'tabelaTodasMovimentacoes'}).findAll('tr')[::-1]):
            linhas = entrada.findAll('td')
            rp = {'ordem': _}
            rp['data'] = datetime.datetime.strptime(linhas[0].get_text().strip(), '%d/%m/%Y').date()

            if linhas[1].find('a'):
                rp['link0'] = linhas[1].find('a').get('href')

            if linhas[2].find('a'):
                rp['link1'] = linhas[2].find('a').get('href')
                rp['movimentacaotag'] =  linhas[2].find('a').get_text().strip()
            else:
                rp['movimentacaotag'] = linhas[2].next.strip()

            rp['movimentacaotag'], x = MovimentacaoTag.objects.get_or_create(nome=rp['movimentacaotag'])

            rp['movimentacaodescricao'] = linhas[2].find('span').get_text()
            coleta.append(rp)
        if len(coleta)>0:
            res['movimentacoes'] = coleta

        res = {k.replace(':','').lower().strip(): v for k,v in res.items()}
        res = {unidecode(k).replace(' ','_').replace('_/_','_'): (v.strip() if isinstance(v, str) else v) for k,v in res.items()}
        
        if 'valor_da_acao' in res.keys():
            res['valor_da_acao'] = float(res['valor_da_acao'].replace('.','').replace(',','.'))

        SegundaInstancia_m = SegundaInstancia(**{k:v for k,v in res.items() if k not in ['partes','numeros_1_instancia', 'movimentacoes' ]})
        SegundaInstancia_m.save()

        if 'partes' in res.keys():
            partes = [{k: (v.lower().strip().replace(':','') if k =='posicao' else v) for k,v in i.items()} for i in res['partes']]
            
            for p in partes:
                p = Parte.objects.get_or_create(nome = p['nome']), p['posicao']
                posicao = Posicao(segunda_instancia=SegundaInstancia_m, parte=p[0][0], posicao=p[1])
                posicao.save()

        if 'numeros_1_instancia' in res.keys():
            n_1_instancia = [{k:v for k,v in i.items() if v !='-'} for i in res['numeros_1_instancia']]
            Numero1Instancia.objects.bulk_create([Numero1Instancia(**i, segunda_instancia=SegundaInstancia_m) for i in n_1_instancia])
        
        if 'movimentacoes' in res.keys():
            m = [Movimentacao(**i, segunda_instancia=SegundaInstancia_m) for i in res['movimentacoes']]
            Movimentacao.objects.bulk_create(m)

    seq += 1
    if vazio == 25:
        break

    print(seq, vazio)
    
    


2202004 0


In [10]:
res['movimentacoes']


In [5]:
# driver.get(r.url)
# sp = bs(r.content, 'html5lib')
coleta = []
for _, entrada in enumerate(sp.find(attrs={'id':'tabelaTodasMovimentacoes'}).findAll('tr')[::-1]):
    linhas = entrada.findAll('td')
    rp = {'ordem': _}
    rp['data'] = datetime.datetime.strptime(linhas[0].get_text().strip(), '%d/%m/%Y').date()
    
    if linhas[1].find('a'):
        rp['link0'] = linhas[1].find('a').get('href')
    
    if linhas[2].find('a'):
        rp['link1'] = linhas[2].find('a').get('href')
        rp['movimentacaotag'] =  linhas[2].find('a').get_text().strip()
    else:
        rp['movimentacaotag'] = linhas[2].next.strip()
    
    rp['movimentacaotag'], x = MovimentacaoTag.objects.get_or_create(nome=rp['movimentacaotag'])
    
    rp['movimentacaodescricao'] = linhas[2].find('span').get_text()
    coleta.append(rp)
    
coleta

[{'ordem': 0,
  'data': datetime.date(2019, 9, 10),
  'movimentacaotag': <MovimentacaoTag: Processo Cadastrado>,
  'movimentacaodescricao': 'SJ 1.2.1 - Serv. de Entrada de Originários do Orgão Especial e Câmara Especial'},
 {'ordem': 1,
  'data': datetime.date(2019, 9, 10),
  'movimentacaotag': <MovimentacaoTag: Processo encaminhado para a Distribuição de Originários>,
  'movimentacaodescricao': ''},
 {'ordem': 2,
  'data': datetime.date(2019, 9, 10),
  'movimentacaotag': <MovimentacaoTag: Distribuição por Sorteio>,
  'movimentacaodescricao': 'Órgão Julgador: 102 - Órgão Especial\nRelator: 10549 - Beretta da Silveira'},
 {'ordem': 3,
  'data': datetime.date(2019, 9, 10),
  'link0': '#?cdDocumento=5',
  'link1': '#?cdDocumento=5',
  'movimentacaotag': <MovimentacaoTag: Conclusos para o Relator (Expedido Termo com Conclusão)>,
  'movimentacaodescricao': 'BERETTA DA SILVEIRA'},
 {'ordem': 4,
  'data': datetime.date(2019, 9, 11),
  'link0': '#?cdDocumento=6',
  'link1': '#?cdDocumento=6',


In [12]:
SegundaInstancia.objects.all().delete()

(38,
 {'core.Numero1Instancia': 4,
  'core.Movimentacao': 21,
  'core.Posicao': 9,
  'core.SegundaInstancia': 4})

In [19]:
from core.models import *


# a = Person(nome='Ciclano', pertinente=True)
# a.save()

# b = Processo(nome='Teste1')
# b.save()

# c = Posicao(person=a, processo=b)
# c.save()

# a = Processo.objects.latest('id')

a

<Processo: Teste1>

In [36]:
res

NameError: name 'res' is not defined

In [2]:
from bs4 import BeautifulSoup as bs
from seleniumrequests import Chrome
import re
from unidecode import unidecode
from core.models import *
import concurrent.futures




# segunda_instancia.objects.all().delete()

(16949,
 {'core.numero_1_instancia': 3988,
  'core.partes': 8946,
  'core.segunda_instancia': 4015})

In [7]:

def calculo_cnj(seq, ano, j, tr, vara):
    vara = str(vara)
    while len(vara) < 4:
        vara = vara + '0'
    n = int(f'{seq}{ano}{j}{tr}{vara}00')
    d = str(98 - n%97)
    if len(d) == 1:
        d = '0' + d
    seq = str(seq)
    while len(seq) < 7:
        seq = '0' + seq
    return f'{seq}-{d}.{ano}.{j}.{tr}.{vara}'

import time

driver = Chrome()

driver.get('https://esaj.tjsp.jus.br/sajcas/login?service=https%3A%2F%2Fesaj.tjsp.jus.br%2Fesaj%2Fj_spring_cas_security_check')
driver.find_element_by_xpath('//*[@id="usernameForm"]').send_keys('01378744055')
driver.find_element_by_xpath('//*[@id="passwordForm"]').send_keys('rasengan')
driver.find_element_by_xpath('//*[@id="pbEntrar"]').click()
driver.get('https://esaj.tjsp.jus.br/cposg/open.do')

seq = 2000002


params = {

'conversationId': '',
'paginaConsulta': '1',
'localPesquisa.cdLocal': '-1',
'cbPesquisa': 'NUMPROC',
'tipoNuProcesso': 'UNIFICADO',
'dePesquisaNuUnificado': calculo_cnj(seq,2019,8,26,0),
'dePesquisa': '',
'uuidCaptcha': '',
}

r = driver.request('GET', 'https://esaj.tjsp.jus.br/cposg/search.do', params=params)
driver.get(r.url)
seq +=1
time.sleep(0.6)



In [20]:
calculo_cnj(seq,2019,8,26,0)

'1288213-20.2019.8.26.0000'

In [15]:
2000523-17.2019.8.26.0000,

<td id="mensagemRetorno" role="alert" tabindex="0">
           <li>Não existem informações disponíveis para os parâmetros informados.</li>
       </td>